## Expert Knowledge Worker

Features:
- A question answering agent that is an expert knowledge worker
- To be used by employees of Insurellm, an Insurance Tech company
- The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

Technology:
- RAG: LangChain
- Embedding model: OpenAIEmbeddings or HuggingFace sentence-transformers
- Encoding method: Auto-encoding
- Vector datastore: Chroma or FAISS
- Vector DB visualization: Plotly
- Dimensionality reduction technique: t-SNE

# Dependencies

In [ ]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Setup

In [ ]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"
db_type = "Chroma"
# db_type = "FAISS"
embed_type = "OpenAIEmbeddings"
# embed_type = "sentence-transformers"

In [ ]:
# Load environment variables

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Create Knowledge Base for RAG

## Load Company Documents

Uses LangChain to read in a Knowledge Base of documents and to divide up documents into overlaping chunks.

In [ ]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("../knowledge-base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

## Vector Embeddings

Convert chunks of text into Vectors using OpenAIEmbeddings and store the Vectors in Chroma (or FAISS).

In [ ]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = None
# OpenAIEmbeddings is OpenAI's vector embedding models
if embed_type == "OpenAIEmbeddings":
    embeddings = OpenAIEmbeddings()

# sentence-transformers is a free Vector embeddings model from HuggingFace
elif embed_type == "sentence-transformers":
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if embeddings is None:
    print("ERROR: embeddings not set. Check embed_type is set to a valid model")

## Create Vector Datastore

In [ ]:
# Create vectorstore
vectorstore = None

# Chroma is a popular open source Vector Database based on SQLLite
if db_type == "Chroma":
    # Delete vector DB if already exists
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
    
    # Create vectorstore
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
    
    print(f"Vectorstore created with {vectorstore._collection.count()} documents")

    # Get one vector and find how many dimensions it has
    collection = vectorstore._collection
    sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
    dimensions = len(sample_embedding)
    print(f"The vectors have {dimensions:,} dimensions")
    
# FAISS is an in-memory vector DB from Facebook
elif db_type == "FAISS":
    # Create vectorstore
    vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
    
    total_vectors = vectorstore.index.ntotal
    dimensions = vectorstore.index.d
    print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

else:
    print("ERROR: Vector datastore not created. Check db_type is set to a valid database")

# Visualizing the Vector Store

Humans are not very good at visualizing things with more than 3 dimensions so to visualize a vector datastore with thousands of dimesions. We need to use techniques like projecting down to reduce the dimensions to only 2 or 3 dimensions in a way that does the best possible job at separating things out to stay faithful to the multi-dimensional representation.

For example, things that are far apart in these multiple dimensions will still be far apart even when projected down to 2 dimensions.

[t-distributed stochastic neighbor embedding (t-SNE)](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) is a nonlinear dimensionality reduction technique for embedding high-dimensional data for visualization in a low-dimensional space of two or three dimensions.

## Configure Visualization

In [ ]:
# Prework
if db_type == "Chroma":
    result = collection.get(include=['embeddings', 'documents', 'metadatas'])
    vectors = np.array(result['embeddings'])
    documents = result['documents']
    doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
    colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

elif db_type == "FAISS":
    vectors = []
    documents = []
    doc_types = []
    colors = []
    color_map = {'products':'blue', 'employees':'green', 'contracts':'red', 'company':'orange'}
    
    for i in range(total_vectors):
        vectors.append(vectorstore.index.reconstruct(i))
        doc_id = vectorstore.index_to_docstore_id[i]
        document = vectorstore.docstore.search(doc_id)
        documents.append(document.page_content)
        doc_type = document.metadata['doc_type']
        doc_types.append(doc_type)
        colors.append(color_map[doc_type])
        
    vectors = np.array(vectors)

else:
    print("ERROR: Vector datastore not created. Check db_type is set to a valid database")

## Reduce the dimensionality to 2D

In [ ]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title=f'2D {db_type} Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Reduce the dimensionality to 3D

In [ ]:
# 3D representation isn't as easy to navigate

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title=f'3D {db_type} Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

# Expert Knowledge Worker

Use LangChain to bring it all together by creating a conversation chain with RAG and memory.

Key abstractions in LangChain:
- LLM: represents abstraction around a model
- Retriever: interface onto somthing like a vector store used for RAG retrieval
- Memory: represents a history of a conversation with a chatbot in memory

Because LangChain abstracts the reprentation of the LLM, retriever and memory the code is the same for any model and knowledge base.

Note: ok to ignore _Deprecation Warning_ for now; LangChain are not expected to remove ConversationBufferMemory any time soon.

## Setup

In [ ]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

## Clear Memory

Clear the memory from the testing and restart conversation chain for UI.

In [ ]:
# clear the memory and restart conversation chain for UI
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Functions

In [ ]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

## UI

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages", examples=["what is insurellm?","what did avery do before?", "does insurellm offer any products in the auto industry space?"], title="Insurellm Expert Knowledge Worker").launch(inbrowser=True)